## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000282829AE1D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000282829AF640>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([
    HumanMessage(content="Hi, My name os Suyash and I am a chief AI Engineer")
])

AIMessage(content="Hello Suyash, it's nice to meet you! \n\nThat's a fascinating role. As a large language model, I'm always interested in learning more about how AI is being developed and used. What kind of projects are you working on? \n\nI'm happy to answer any questions you have about language models or AI in general.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 24, 'total_tokens': 101, 'completion_time': 0.14, 'prompt_time': 0.002267275, 'queue_time': 0.232055454, 'total_time': 0.142267275}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0bd9c45a-e4f3-427b-aabc-dfb1acba9574-0', usage_metadata={'input_tokens': 24, 'output_tokens': 77, 'total_tokens': 101})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Suyash and I am a chief AI Engineer"),
        AIMessage(content="Hello Suyash,\n\nIt's a pleasure to meet you! Being a Chief AI Engineer is a fascinating role. What kind of projects are you currently working on?  \n\nI'm eager to learn more about your work in the field of AI. Perhaps you could tell me about:\n\n* **The most challenging AI project you've tackled**\n* **Your favorite AI tool or framework**\n* **The biggest impact you hope to make with AI**\n\n\nI'm always interested in hearing from experienced professionals like yourself.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Suyash, and you are a Chief AI Engineer!  \n\nI remember our introduction. 😊 \n\nIs there anything else you'd like to talk about or discuss?  I'm here to help in any way I can.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 157, 'total_tokens': 211, 'completion_time': 0.098181818, 'prompt_time': 0.006793916, 'queue_time': 0.23235377299999999, 'total_time': 0.104975734}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-174624d7-e2ab-46dd-80c8-cedd3a9155b1-0', usage_metadata={'input_tokens': 157, 'output_tokens': 54, 'total_tokens': 211})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [10]:
config={"configurable":{"session_id":"Chat1"}}

In [12]:
response=with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Suyash and I am a chief AI Engineer")
    ],
    config=config
)

In [13]:
response.content

"It's a pleasure to meet you, Suyash! \n\nBeing a Chief AI Engineer is a very important role. What kind of work are you focusing on these days?  I'm always interested in learning more about the diverse applications of AI. \n\n"

In [14]:
with_message_history.invoke(
    [
        HumanMessage(content="What is my name?")
    ],
    config=config
)

AIMessage(content='Your name is Suyash.  😊  \n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 177, 'total_tokens': 201, 'completion_time': 0.043636364, 'prompt_time': 0.008706602, 'queue_time': 0.234707366, 'total_time': 0.052342966}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0376172e-bb38-45f5-9427-2a3c43ba588b-0', usage_metadata={'input_tokens': 177, 'output_tokens': 24, 'total_tokens': 201})

In [16]:
## change the config-->changing session id
config1={"configurable":{"session_id":"Chat2"}}
response=with_message_history.invoke(
    [
        HumanMessage(content="What is my name?")
    ],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to use it!\n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.\n\nWhat can I do for you today? 😊  \n"

In [18]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John!  \n\nI remember that you told me earlier. 😊  Anything else I can help you with, John?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [25]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Suyash")]})

AIMessage(content="Hi Suyash, it's nice to meet you!\n\nI'm happy to help with any questions you have. What can I do for you today? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 32, 'total_tokens': 71, 'completion_time': 0.070909091, 'prompt_time': 0.002374085, 'queue_time': 0.232015884, 'total_time': 0.073283176}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-479d7007-29a7-45f0-abba-dc9b67eafee6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 39, 'total_tokens': 71})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [26]:
config={"configurable":{"session_id":"Chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Suyash")],
    config=config
)

response.content

"Hi Suyash! It's great to meet you too. 👋 \n\nWhat can I do for you today? 🤔  \n"

In [27]:
##Add more complexity

prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the question to the best of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain= prompt | model

In [28]:
response= chain.invoke({"messages":[HumanMessage(content="Hi My name is Suyash")], "language":"Hindi"})
response.content

'नमस्ते सुयश!  😊  \n\nक्या मैं कुछ करने में आपकी मदद कर सकता हूँ? आप मुझसे कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी क्षमता के अनुसार उत्तर दूंगा। \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [29]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [31]:
config={"configurable":{"session_id":"Chat4"}}
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Suyash")], "language":"Hindi"},
    config=config
)
response.content

'नमस्ते सुयाश! 😊  \n\nआप कैसे हो? \n'

In [32]:
response=with_message_history.invoke(
    {'messages': [HumanMessage(content="Whats my name?")], "language":"Hindi"},
    config=config
)
response.content

'आपका नाम सुयाश है।  😄 \n\nमुझे यह बताते हुए बहुत खुशी हो रही है! \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [33]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens=70,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

e:\GenAI_Krish\Code\venv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Suyash Mohanty\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model

)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do I like")],
    "language": "English"
    }
)

response.content

'You said you like vanilla ice cream! 😊  \n\n\n\nIs there anything else I can help you with?\n'

In [35]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked: "What\'s 2 + 2?"\n'

In [36]:
## Lets wrap this in the MEssage History
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [37]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

'Your name is Bob! 😊 \n\n\n\n'

In [38]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

'You asked "What\'s 2 + 2?"  \n'